### Setup

In [0]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive











### Import tables

In [0]:
DATA_PATH = r'/content/drive/My Drive/Data'
analysis_path = os.path.join(DATA_PATH, 'Tables', 'Analysis.csv')
dive_path = os.path.join(DATA_PATH, 'Tables', 'Dives.csv')
site_path = os.path.join(DATA_PATH, 'Tables', 'Sites.csv')
imagetable_path = os.path.join(DATA_PATH, 'Tables', 'ImageTable.csv')
quadratvideos_path = os.path.join(DATA_PATH, 'Tables', 'QuadratVideos.csv')
quadratstills_path = os.path.join(DATA_PATH, 'Tables', 'QuadratStills.csv')
fullvideos_path = os.path.join(DATA_PATH, 'Tables', 'FullVideos.csv')

In [0]:
analysis_df = pd.read_csv(analysis_path, index_col=0,
                          dtype={'Count':float})
dive_df = pd.read_csv(dive_path, index_col=0, parse_dates=['Date'])
site_df = pd.read_csv(site_path, index_col=0)
image_df = pd.read_csv(imagetable_path, index_col=0)
quadrat_df = pd.read_csv(quadratvideos_path, index_col=0)
still_df = pd.read_csv(quadratstills_path, index_col=0)
video_df = pd.read_csv(fullvideos_path, index_col=0)

In [0]:
site_df.rename(columns={'Name':'Site Name'}, inplace=True)

In [6]:
analysis_df = analysis_df.set_index('Analysis Index')
analysis_df

Dive Index  Quadrat  ...       4mm       2mm
Analysis Index                       ...                    
0                        0        1  ...       NaN       NaN
1                        0        2  ...       NaN       NaN
2                        0        3  ...  0.234848  0.340909
3                        1        1  ...  0.088836  0.116447
4                        1        2  ...  0.035549  0.037094
...                    ...      ...  ...       ...       ...
1603                   534        2  ...  0.170213  0.053191
1604                   534        3  ...  0.104738  0.094763
1605                   535        1  ...  0.160243  0.248479
1606                   535        2  ...  0.095302  0.351678
1607                   535        3  ...  0.086957  0.043478

[1608 rows x 14 columns]

In [0]:
dive_df.loc[dive_df['Shale (%)'].isnull(), 'Shale (%)'] = 0
dive_df.to_csv(dive_path)

### Merge analysis, dive and site tables into a single long table

In [8]:
data_df = pd.merge(analysis_df, dive_df, on='Dive Index', how='left')
data_df.index.name = 'Analysis Index'
data_df

Dive Index  ...  Underlying Substrate Depth (cm)
Analysis Index              ...                                 
0                        0  ...                              NaN
1                        0  ...                              NaN
2                        0  ...                              NaN
3                        1  ...                              NaN
4                        1  ...                              NaN
...                    ...  ...                              ...
1603                   534  ...                              NaN
1604                   534  ...                              NaN
1605                   535  ...                              NaN
1606                   535  ...                              NaN
1607                   535  ...                              NaN

[1608 rows x 31 columns]

In [9]:
data_df = pd.merge(data_df, site_df, on='PSN', how='left')
data_df.index.name = 'Analysis Index'
data_df

Dive Index  Quadrat  ...     Site Name     Lake
Analysis Index                       ...                       
0                        0        1  ...       Cobourg  Ontario
1                        0        2  ...       Cobourg  Ontario
2                        0        3  ...       Cobourg  Ontario
3                        1        1  ...       Cobourg  Ontario
4                        1        2  ...       Cobourg  Ontario
...                    ...      ...  ...           ...      ...
1603                   534        2  ...  Mohawk Point     Erie
1604                   534        3  ...  Mohawk Point     Erie
1605                   535        1  ...  Mohawk Point     Erie
1606                   535        2  ...  Mohawk Point     Erie
1607                   535        3  ...  Mohawk Point     Erie

[1608 rows x 35 columns]

In [10]:
data_image_df = pd.merge(image_df, data_df, on='Analysis Index', how='left')
data_image_df

Analysis Index                                Name  ...   Site Name     Lake
0               0.0  GLNI_3554-1_2018-06-19_image-1.nef  ...     Cobourg  Ontario
1               0.0  GLNI_3554-1_2018-06-19_image-2.nef  ...     Cobourg  Ontario
2               1.0  GLNI_3554-2_2018-06-19_image-2.nef  ...     Cobourg  Ontario
3               1.0  GLNI_3554-2_2018-06-19_image-1.nef  ...     Cobourg  Ontario
4               2.0  GLNI_3554-3_2018-06-19_image-2.nef  ...     Cobourg  Ontario
..              ...                                 ...  ...         ...      ...
916          1330.0  GLNI_1274-2_2016-07-20_image-4.nef  ...  Rock Point     Erie
917          1331.0  GLNI_1274-3_2016-07-20_image-3.nef  ...  Rock Point     Erie
918          1331.0  GLNI_1274-3_2016-07-20_image-4.nef  ...  Rock Point     Erie
919          1329.0  GLNI_1274-1_2016-07-20_image-3.nef  ...  Rock Point     Erie
920          1329.0  GLNI_1274-1_2016-07-20_image-4.nef  ...  Rock Point     Erie

[921 rows x 39 columns]

In [11]:
simplified_image_df = data_image_df.loc[(data_image_df['Name'].notnull())&(data_image_df['Analysis Index'].notnull())].sort_values('Timestamp').groupby(by='Analysis Index').last()
simplified_image_df

Name  ...     Lake
Analysis Index                                      ...         
0.0             GLNI_3554-1_2018-06-19_image-2.nef  ...  Ontario
1.0             GLNI_3554-2_2018-06-19_image-2.nef  ...  Ontario
2.0             GLNI_3554-3_2018-06-19_image-2.nef  ...  Ontario
3.0             GLNI_3553-1_2018-06-19_image-2.nef  ...  Ontario
4.0             GLNI_3553-2_2018-06-19_image-2.nef  ...  Ontario
...                                            ...  ...      ...
1603.0          GLNI_1346-2_2017-08-17_image-1.nef  ...     Erie
1604.0          GLNI_1346-3_2017-08-17_image-1.nef  ...     Erie
1605.0          GLNI_1345-1_2017-08-17_image-2.nef  ...     Erie
1606.0          GLNI_1345-2_2017-08-17_image-2.nef  ...     Erie
1607.0          GLNI_1345-3_2017-08-17_image-2.nef  ...     Erie

[489 rows x 38 columns]

In [12]:
data_video_df = pd.merge(data_df, quadrat_df, on=['Analysis Index'], how='right')
data_video_df

Analysis Index  ...                                 Quadrat Video Path
0               180  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
1               181  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
2               182  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
3               183  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
4               184  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
..              ...  ...                                                ...
727            1598  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
728            1602  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
729            1603  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
730            1604  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
731            1607  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...

[732 rows x 39 columns]

In [13]:
simplified_video_df = data_video_df.loc[data_video_df['Name'].notnull()\
                                    & data_video_df['Analysis Index'].notnull()]\
                                    .sort_values('Name').groupby(by='Analysis Index').last()
simplified_video_df

Dive Index  ...                                 Quadrat Video Path
Analysis Index              ...                                                   
180                     60  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
181                     60  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
182                     60  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
183                     61  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
184                     61  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
...                    ...  ...                                                ...
1598                   532  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
1602                   534  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
1603                   534  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
1604                   534  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
1607                   535  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...

[718 rows x 38 columns]

In [14]:
data_still_df = pd.merge(data_df, still_df, on=['Analysis Index'], how='right')
data_still_df

Analysis Index  ...                                 Quadrat Still Path
0              798  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
1              798  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
2              798  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
3              798  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
4              799  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
..             ...  ...                                                ...
62            1291  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
63            1291  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
64            1292  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
65            1292  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...
66            1292  ...  G:\MusselsCurated\GLNI_2012-2018\GLNI_quadrats...

[67 rows x 39 columns]

In [0]:
simplified_still_df = data_still_df.loc[data_still_df['Name'].notnull()\
                                    & data_still_df['Analysis Index'].notnull()]\
                                    .sort_values('Name').groupby(by='Analysis Index').last()

In [16]:
len(simplified_still_df)

27

In [17]:
simplified_merged_df = simplified_video_df.append(simplified_image_df).append(simplified_still_df)
simplified_merged_df

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


10mm  ...  Underlying Substrate Type
Analysis Index            ...                           
180.0           0.108000  ...                        NaN
181.0           0.228814  ...                        NaN
182.0           0.208333  ...                        NaN
183.0           0.410853  ...                        NaN
184.0           0.449640  ...                        NaN
...                  ...  ...                        ...
1234.0          0.169538  ...                    Bedrock
1235.0          0.183131  ...                    Bedrock
1290.0               NaN  ...               Glacial Till
1291.0               NaN  ...               Glacial Till
1292.0               NaN  ...               Glacial Till

[1234 rows x 41 columns]

In [18]:
simplified_merged_df = simplified_merged_df.loc[simplified_merged_df['Raw Image Path'].notnull()|\
              (simplified_merged_df['Raw Image Path'].isnull()&simplified_merged_df['Quadrat Video Path'].notnull())|\
              (simplified_merged_df['Raw Image Path'].isnull()&simplified_merged_df['Quadrat Video Path'].isnull())]
simplified_merged_df

10mm  ...  Underlying Substrate Type
Analysis Index            ...                           
180.0           0.108000  ...                        NaN
181.0           0.228814  ...                        NaN
182.0           0.208333  ...                        NaN
183.0           0.410853  ...                        NaN
184.0           0.449640  ...                        NaN
...                  ...  ...                        ...
1234.0          0.169538  ...                    Bedrock
1235.0          0.183131  ...                    Bedrock
1290.0               NaN  ...               Glacial Till
1291.0               NaN  ...               Glacial Till
1292.0               NaN  ...               Glacial Till

[1234 rows x 41 columns]

In [19]:
simplified_merged_df.columns

Index(['10mm', '12.5mm', '14mm', '16mm', '2mm', '4mm', '6.3mm', '8mm',
       'Bedrock (%)', 'Biomass', 'Boulders (%)', 'CSN', 'Clay (%)',
       'Cobble (%)', 'Count', 'Cruise #', 'Date', 'Depth (m)', 'Dive Index',
       'Empty Coverage', 'Gravel (%)', 'Lake', 'Latitude', 'Live Coverage',
       'Longitude', 'Name', 'Overall Coverage', 'PSN', 'Quadrat',
       'Quadrat Still Path', 'Quadrat Video Path', 'Raw Image Path',
       'Rock (%)', 'Sand (%)', 'Shale (%)', 'Silt (%)', 'Site Name',
       'Source Video', 'Timestamp', 'Underlying Substrate Depth (cm)',
       'Underlying Substrate Type'],
      dtype='object')

In [0]:
# Data pre-processing: use overall coverage when live coverage is missing
simplified_merged_df.loc[simplified_merged_df['Live Coverage'].isnull(), 'Live Coverage'] = \
    simplified_merged_df.loc[simplified_merged_df['Live Coverage'].isnull(), 'Overall Coverage'].values

In [0]:
# Data pre-processing: transform percentage size distribution to total count for each category
simplified_merged_df['Count'] = simplified_merged_df['Count'].astype(float)
simplified_merged_df['Biomass'] = simplified_merged_df['Biomass'].astype(float)
simplified_merged_df['16mm'] = simplified_merged_df['16mm'].astype(float).fillna(0.)*simplified_merged_df['Count']
simplified_merged_df['14mm'] = simplified_merged_df['14mm'].astype(float).fillna(0.)*simplified_merged_df['Count']
simplified_merged_df['12.5mm'] = simplified_merged_df['12.5mm'].astype(float).fillna(0.)*simplified_merged_df['Count']
simplified_merged_df['10mm'] = simplified_merged_df['10mm'].astype(float).fillna(0.)*simplified_merged_df['Count']
simplified_merged_df['8mm'] = simplified_merged_df['8mm'].astype(float).fillna(0.)*simplified_merged_df['Count']
simplified_merged_df['6.3mm'] = simplified_merged_df['6.3mm'].astype(float).fillna(0.)*simplified_merged_df['Count']
simplified_merged_df['4mm'] = simplified_merged_df['4mm'].astype(float).fillna(0.)*simplified_merged_df['Count']
simplified_merged_df['2mm'] = simplified_merged_df['2mm'].astype(float).fillna(0.)*simplified_merged_df['Count']

In [22]:
simplified_merged_df = simplified_merged_df[['10mm', '12.5mm', '14mm', '16mm', '2mm', '4mm', '6.3mm', '8mm',
       'Biomass', 'Count', 'Date', 'Dive Index',
       'Empty Coverage', 'Lake', 'Latitude', 'Live Coverage',
       'Longitude', 'Name', 'PSN', 'Quadrat']]
simplified_merged_df['Analysis Index'] = simplified_merged_df.index

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [23]:
simplified_merged_df.columns

Index(['10mm', '12.5mm', '14mm', '16mm', '2mm', '4mm', '6.3mm', '8mm',
       'Biomass', 'Count', 'Date', 'Dive Index', 'Empty Coverage', 'Lake',
       'Latitude', 'Live Coverage', 'Longitude', 'Name', 'PSN', 'Quadrat',
       'Analysis Index'],
      dtype='object')

In [0]:
simplified_merged_df.to_csv(os.path.join(DATA_PATH, 'Tables', 'SimplifiedImagingAnalysis.csv'))

In [25]:
merged_df = pd.merge(image_df.append(still_df).append(quadrat_df), data_df, on='Analysis Index', how='left')
merged_df

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Analysis Index                                Name  ...     Site Name     Lake
0                0.0  GLNI_3554-1_2018-06-19_image-1.nef  ...       Cobourg  Ontario
1                0.0  GLNI_3554-1_2018-06-19_image-2.nef  ...       Cobourg  Ontario
2                1.0  GLNI_3554-2_2018-06-19_image-2.nef  ...       Cobourg  Ontario
3                1.0  GLNI_3554-2_2018-06-19_image-1.nef  ...       Cobourg  Ontario
4                2.0  GLNI_3554-3_2018-06-19_image-2.nef  ...       Cobourg  Ontario
...              ...                                 ...  ...           ...      ...
1715          1598.0  GLNI_1348-3_2017-08-17_video-1.mp4  ...  Mohawk Point     Erie
1716          1602.0  GLNI_1346-1_2017-08-17_video-1.mp4  ...  Mohawk Point     Erie
1717          1603.0  GLNI_1346-2_2017-08-17_video-1.mp4  ...  Mohawk Point     Erie
1718          1604.0  GLNI_1346-3_2017-08-17_video-1.mp4  ...  Mohawk Point     Erie
1719          1607.0  GLNI_1345-3_2017-08-17_video-1.mp4  ...  Mohawk Point     Erie

[1720 rows x 42 columns]

In [0]:
merged_df.to_csv(os.path.join(DATA_PATH, 'Tables', 'MergedTable.csv'))